In [26]:
import numpy as np
import statsmodels.api as sm
import pandas as pd 
import sklearn.cross_validation as cv
from sklearn.linear_model import LassoCV,RidgeCV,LinearRegression,SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import os 
import statsmodels.formula.api as sm

In [14]:
## Setting up path file
os.chdir(r"C:\Users\siddh\Desktop\analytics\ECEN 689\Challenge4 Wine")

In [15]:
## Read in files
df_train = pd.read_csv("winequality-white-training.csv").drop(['Id'],axis = 1)
df_test = pd.read_csv("winequality-white-testing.csv").drop(['Id'],axis = 1)
df_test2 = pd.read_csv("winequality-red-testing.csv").drop(['Id'],axis = 1)

In [16]:
def pred_target(df,colname):
    X = df.drop([colname],axis = 1)
    Y = df[colname]
    return X,Y

In [17]:
X_train,Y_train = pred_target(df_train,"quality")
X_test = df_test
x_val = df_test2

In [77]:
## Cross validation with Lasso - approach 1 to predict white wine quality 
alphas = np.logspace(-4, 2, 2000)
scores = np.empty_like(alphas)

lassocv = LassoCV(alphas = alphas ,cv = 5, max_iter = 1000000,fit_intercept=False)
lassocv.fit(X_train, Y_train)

lasso = Lasso(fit_intercept = False,max_iter = 1000000)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, Y_train)

predictions = lasso.predict(X_test)
predictions = pd.DataFrame(predictions,columns = ["quality"])
predictions.to_csv("linreg_pred_white2.csv")

In [11]:
#X_train = sm.add_constant(X_train) # adding a constant
## Printing out p values to find predictor p values 
model = sm.OLS(Y_train, X_train).fit()
predictions = model.predict(X_train) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                 2.146e+04
Date:                Tue, 16 Oct 2018   Prob (F-statistic):               0.00
Time:                        14:03:45   Log-Likelihood:                -4444.5
No. Observations:                3898   AIC:                             8911.
Df Residuals:                    3887   BIC:                             8980.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
fixed acidity           -0.0522 

In [20]:
## OLS Regression - Approach 2 "Used for Submission"
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train,Y_train)
new_pred = lr.predict(X_test)
new_pred = pd.DataFrame(new_pred,columns = ["quality"])
new_pred.to_csv("linreg_pred_white_with_inter.csv")

In [21]:
## Predict red wine rating with white coefficients
predictions1 = lr.predict(x_val)
predictions1 = pd.DataFrame(predictions1,columns = ["quality"])
predictions1.to_csv("linreg_pred_white_red_3.csv")

In [24]:
coefficients = pd.DataFrame(lr.coef_)
coefficients.to_csv("White_wine_coefficients_1.csv")

In [39]:
## SGD Regressor - Approach 3
sg = SGDRegressor(alpha = 0.000001) ##Thumb rule, source:"http://sdsawtelle.github.io/blog/output/week2-andrew-ng-machine-learning-with-python.html
sg.max_iter = np.ceil(np.ceil(10**6 / len(Y_train.values)))
sg.fit(X_train.values,Y_train.values)
sgd_white = sg.predict(X_test.values)

In [40]:
## Predict Red with white wine coeffients from SGD
sgd_white_red = sg.predict(x_val)
predictions_red_sgd = pd.DataFrame(sgd_white_red,columns = ['quality'])
predictions_red_sgd.to_csv("linreg_pred_white_red_sgd.csv")